In [1]:
from pyspark.sql import SparkSession
import os

__file__ = '/home/vk/Spark_Source_Code/net.jgp.books.spark.ch01/data'
current_dir = os.path.dirname(__file__)
relative_path = __file__ + "/books.csv"
absolute_file_path = os.path.join(current_dir, relative_path)

# Creates a session on a local master
session = SparkSession.builder.appName("CSV to Dataset").master("local[*]").getOrCreate()

# Reads a CSV file with header, called books.csv, stores it in a dataframe
df = session.read.csv(header=True, inferSchema=True, path=absolute_file_path)

# Shows at most 5 rows from the dataframe
df.show(5)

# Good to stop SparkSession at the end of the application
session.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/17 11:26:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+--------+--------------------+-----------+--------------------+
| id|authorId|               title|releaseDate|                link|
+---+--------+--------------------+-----------+--------------------+
|  1|       1|Fantastic Beasts ...|   11/18/16|http://amzn.to/2k...|
|  2|       1|Harry Potter and ...|    10/6/15|http://amzn.to/2l...|
|  3|       1|The Tales of Beed...|    12/4/08|http://amzn.to/2k...|
|  4|       1|Harry Potter and ...|    10/4/16|http://amzn.to/2k...|
|  5|       2|Informix 12.10 on...|    4/23/17|http://amzn.to/2i...|
+---+--------+--------------------+-----------+--------------------+
only showing top 5 rows



In [1]:
'''
  CSV to a relational database.

  @author rambabu.posa
'''

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

__file__ = '/home/vk/Spark_Source_Code/net.jgp.books.spark.ch02/data'
current_dir = os.path.dirname(__file__)
relative_path = __file__ + "/authors.csv"
absolute_file_path = os.path.join(current_dir, relative_path)

# Creates a session on a local master
spark = SparkSession.builder.appName("CSV to DB").master("local").getOrCreate()

#  Step 1: Ingestion
#  ---------
#
#  Reads a CSV file with header, called authors.csv, stores it in a dataframe
df = spark.read.csv(header=True, inferSchema=True, path=absolute_file_path)

# Step 2: Transform
# ---------
# Creates a new column called "name" as the concatenation of lname, a
# virtual column containing ", " and the fname column
df = df.withColumn("name", F.concat(F.col("lname"), F.lit(", "), F.col("fname")))

# Step 3: Save
# ----
#
# The connection URL, assuming your PostgreSQL instance runs locally on the
# default port, and the database we use is "spark_labs"
dbConnectionUrl = "jdbc:postgresql://localhost:5432/spark_labs"

# Properties to connect to the database, the JDBC driver is part of our pom.xml
prop = {"driver":"org.postgresql.Driver", "user":"vadim", "password":"1"}

# Write in a table called ch02
df.write.jdbc(mode='overwrite', url=dbConnectionUrl, table="ch02", properties=prop)

# Good to stop SparkSession at the end of the application
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/17 19:35:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling o41.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:593)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:526)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)
	at org

25/01/17 19:35:51 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [20]:
#@@@@@@@@@@@@@@@@@@@@@ ch03 @@@@@@@@@@@@@@@@@@@@@@@

"""
  CSV ingestion in a dataframe and manipulation.

  @author rambabu.posa
"""
import os
import logging
from pyspark.sql import SparkSession
#from pyspark.sql.functions import F
import pyspark.sql.functions as F

def get_absolute_file_path(path, filename):
    # To get absolute path for a given filename
    file = '/home/vk/Spark_Source_Code/net.jgp.books.spark.ch03/data/'
    current_dir = os.path.dirname(file)
    relative_path = "{}{}".format(path, filename)
    absolute_file_path = os.path.join(current_dir, relative_path)
    return absolute_file_path

def main(spark):
    # The processing code.
    filename = 'Restaurants_in_Wake_County_NC.csv'
    path = ""
    absolute_file_path = get_absolute_file_path(path, filename)

    # Reads a CSV file with header, called
    # Restaurants_in_Wake_County_NC.csv,
    # stores it in a dataframe
    # Creates a session on a local master
  #  spark = SparkSession.builder.appName("CSV to DB").master("local").getOrCreate()
    df = spark.read.csv(header=True, inferSchema=True,path=absolute_file_path)
    
    logging.warning("Type of df is {}".format(type(df)))
    logging.warning("*** Right after ingestion")
    
    df.show(5)
    df.printSchema()
    
    logging.warning("We have {} records.".format(df.count()))

    # Let's transform our dataframe
    df =  df.withColumn("county", F.lit("Wake")) \
            .withColumnRenamed("HSISID", "datasetId") \
            .withColumnRenamed("NAME", "name") \
            .withColumnRenamed("ADDRESS1", "address1") \
            .withColumnRenamed("ADDRESS2", "address2") \
            .withColumnRenamed("CITY", "city") \
            .withColumnRenamed("STATE", "state") \
            .withColumnRenamed("POSTALCODE", "zip") \
            .withColumnRenamed("PHONENUMBER", "tel") \
            .withColumnRenamed("RESTAURANTOPENDATE", "dateStart") \
            .withColumnRenamed("FACILITYTYPE", "type") \
            .withColumnRenamed("X", "geoX") \
            .withColumnRenamed("Y", "geoY") \
            .drop("OBJECTID", "PERMITID", "GEOCODESTATUS")

    df = df.withColumn("id",
                       F.concat(F.col("state"), F.lit("_"),
                                F.col("county"), F.lit("_"),
                                F.col("datasetId")))

    # Shows at most 5 rows from the dataframe
    logging.warning("*** Dataframe transformed")
    df.show(5)
    
    df = df.repartition(4)
    # logging.warning("RDD : {}".format(df.rdd.count()))  ---> 3440 records
    # print("json : {}".format(df.toJSON().collect())) #  ---> JSON

if __name__ == '__main__':
    # Creates a session on a local master
    spark = SparkSession.builder.appName("Restaurants in Wake County, NC") \
        .master("local[*]").getOrCreate()

    # Comment this line to see full log
    spark.sparkContext.setLogLevel('warn')
    main(spark)
    spark.stop()

+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|    HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER| RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|    1001|4092016024|                WABA|2502 1/2 HILLSBOR...|    NULL|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 04:00:00|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    NULL|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 04:00:00|       Food Stand|    6953|-78.78211173|35.73717591|

In [1]:
"""
   Introspection of a schema.ch03

   @author rambabu.posa
"""
import os
import json
import logging
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import sys as sys

def get_absolute_file_path(path, filename):
        # To get absolute path for a given filename
        current_dir = os.path.dirname("")
        relative_path = "{}{}".format(path, filename)
        absolute_file_path = os.path.join(current_dir, relative_path)
        return absolute_file_path

def main(spark):
        # The processing code.
        filename = 'Restaurants_in_Wake_County_NC.csv'
        #path = '../../../../data/'
        path = '/home/vk/Spark_Source_Code/net.jgp.books.spark.ch03/data/'
        absolute_file_path = get_absolute_file_path(path, filename)
        # Reads a CSV file with header, called
        # Restaurants_in_Wake_County_NC.csv,
        # stores it in a dataframe
        df = spark.read.csv(header=True, inferSchema=True, path=absolute_file_path)
        
        df.show(5)
        df.printSchema()

        # Let's transform our dataframe
        df = df.withColumn("county", F.lit("Wake")) \
                .withColumnRenamed("HSISID", "datasetId") \
                .withColumnRenamed("NAME", "name") \
                .withColumnRenamed("ADDRESS1", "address1") \
                .withColumnRenamed("ADDRESS2", "address2") \
                .withColumnRenamed("CITY", "city") \
                .withColumnRenamed("STATE", "state") \
                .withColumnRenamed("POSTALCODE", "zip") \
                .withColumnRenamed("PHONENUMBER", "tel") \
                .withColumnRenamed("RESTAURANTOPENDATE", "dateStart") \
                .withColumnRenamed("FACILITYTYPE", "type") \
                .withColumnRenamed("X", "geoX") \
                .withColumnRenamed("Y", "geoY") \
                .drop("OBJECTID", "PERMITID", "GEOCODESTATUS")

        df = df.withColumn("id",
                F.concat(F.col("state"), F.lit("_"), F.col("county"), F.lit("_"), F.col("datasetId")))

        logging.warning("*** Schema as a tree:")
        print(df.printSchema())

        logging.warning("*** Schema as string: {}".format(df.schema))
        schemaAsJson = df.schema.json()
        parsedSchemaAsJson = json.loads(schemaAsJson)

        logging.warning("*** Schema as JSON: {}".format(json.dumps(parsedSchemaAsJson, indent=2)))

if __name__ == '__main__':
        # Creates a session on a local master
        spark = SparkSession.builder \
                .appName("Schema introspection for restaurants in Wake County, NC") \
                .master("local[*]").getOrCreate()

        # Comment this line to see full log
        spark.sparkContext.setLogLevel('warn')
        main(spark)
        # Good to stop SparkSession at the end of the application
        spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/17 09:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|OBJECTID|    HSISID|                NAME|            ADDRESS1|ADDRESS2|       CITY|STATE|POSTALCODE|   PHONENUMBER| RESTAURANTOPENDATE|     FACILITYTYPE|PERMITID|           X|          Y|GEOCODESTATUS|
+--------+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+--------+------------+-----------+-------------+
|    1001|4092016024|                WABA|2502 1/2 HILLSBOR...|    NULL|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 04:00:00|       Restaurant|    6952|-78.66818477|35.78783803|            M|
|    1002|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    NULL|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 04:00:00|       Food Stand|    6953|-78.78211173|35.73717591|

  "fields": [
    {
      "metadata": {},
      "name": "datasetId",
      "nullable": true,
      "type": "long"
    },
    {
      "metadata": {},
      "name": "name",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "address1",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "address2",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "city",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "state",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "zip",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "tel",
      "nullable": true,
      "type": "string"
    },
    {
      "metadata": {},
      "name": "dateStart",
      "nullable": true,
      "type": "timestamp"
    },
    {
      "metadata": {},
    

root
 |-- datasetId: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address1: string (nullable = true)
 |-- address2: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- dateStart: timestamp (nullable = true)
 |-- type: string (nullable = true)
 |-- geoX: double (nullable = true)
 |-- geoY: double (nullable = true)
 |-- county: string (nullable = false)
 |-- id: string (nullable = true)

None


In [23]:
"""
   CSV ingestion in a dataframe. ch220

   @author rambabu.posa
"""
import os
import logging
from pyspark.sql import SparkSession
#from pyspark.sql.functions import F
import pyspark.sql.functions as F
import json

def get_absolute_file_path(path, filename):
        # To get absolute path for a given filename
        #current_dir = os.path.dirname(__file__)
        current_dir = '/home/vk/Spark_Source_Code/net.jgp.books.spark.ch03/src/main/python/lab220_json_ingestion_schema_manipulation/'
        relative_path = "{}{}".format(path, filename)
        absolute_file_path = os.path.join(current_dir, relative_path)
        return absolute_file_path

def main(spark):
        # The processing code.
        filename = 'Restaurants_in_Durham_County_NC.json'
        path = '../../../../data/'
        absolute_file_path = get_absolute_file_path(path, filename)

        # Reads a JSON file called Restaurants_in_Durham_County_NC.json, stores
        # it in a dataframe
        df = spark.read.json(absolute_file_path)
        logging.warning("*** Right after ingestion")
        df.show(5)
        df.printSchema()
        logging.warning("We have {} records.".format(df.count))

        df =  df.withColumn("county", F.lit("Durham")) \
                .withColumn("datasetId", F.col("fields.id")) \
                .withColumn("name", F.col("fields.premise_name")) \
                .withColumn("address1", F.col("fields.premise_address1")) \
                .withColumn("address2", F.col("fields.premise_address2")) \
                .withColumn("city", F.col("fields.premise_city")) \
                .withColumn("state", F.col("fields.premise_state")) \
                .withColumn("zip", F.col("fields.premise_zip")) \
                .withColumn("tel", F.col("fields.premise_phone")) \
                .withColumn("dateStart", F.col("fields.opening_date")) \
                .withColumn("dateEnd", F.col("fields.closing_date")) \
                .withColumn("type", F.split(F.col("fields.type_description"), " - ").getItem(1)) \
                .withColumn("geoX", F.col("fields.geolocation").getItem(0)) \
                .withColumn("geoY", F.col("fields.geolocation").getItem(1))

        df = df.withColumn("id", F.concat(F.col("state"), F.lit("_"),
                                          F.col("county"), F.lit("_"),
                                          F.col("datasetId")))

        logging.warning("*** Dataframe transformed")
        df.select('id',"state", "county", "datasetId", "name", "address1").show(5)
        df.printSchema()

        logging.warning("*** Looking at partitions")
        partitionCount = df.rdd.getNumPartitions()
        logging.warn("Partition count before repartition: {}".format(partitionCount))

        df = df.repartition(4)
        logging.warning("Partition count after repartition: {}".format(df.rdd.getNumPartitions()))

if __name__ == '__main__':
        # Creates a session on a local master
        spark = SparkSession.builder.appName("Restaurants in Durham County, NC") \
                .master("local[*]").getOrCreate()

        # Comment this line to see full log
        spark.sparkContext.setLogLevel('warn')
        main(spark)
        spark.stop()

+----------------+--------------------+--------------------+--------------------+--------------------+
|       datasetid|              fields|            geometry|    record_timestamp|            recordid|
+----------------+--------------------+--------------------+--------------------+--------------------+
|restaurants-data|{NULL, Full-Servi...|{[-78.9573299, 35...|2017-07-13T09:15:...|1644654b953d1802c...|
|restaurants-data|{NULL, Nursing Ho...|{[-78.8895483, 36...|2017-07-13T09:15:...|93573dbf8c9e799d8...|
|restaurants-data|{NULL, Fast Food ...|{[-78.9593263, 35...|2017-07-13T09:15:...|0d274200c7cef50d0...|
|restaurants-data|{NULL, Full-Servi...|{[-78.9060312, 36...|2017-07-13T09:15:...|cf3e0b175a6ebad2a...|
|restaurants-data|{NULL, NULL, [36....|{[-78.9135175, 36...|2017-07-13T09:15:...|e796570677f7c39cc...|
+----------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

root
 |-- datasetid: string (nullable = true)
 |

/tmp/ipykernel_13767/1294692902.py:60: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn("Partition count before repartition: {}".format(partitionCount))


+---------------+-----+------+---------+--------------------+--------------------+
|             id|state|county|datasetId|                name|            address1|
+---------------+-----+------+---------+--------------------+--------------------+
|NC_Durham_56060|   NC|Durham|    56060|    WEST 94TH ST PUB| 4711 HOPE VALLEY RD|
|NC_Durham_58123|   NC|Durham|    58123|BROOKDALE DURHAM IFS|4434 BEN FRANKLIN...|
|NC_Durham_70266|   NC|Durham|    70266|       SMOOTHIE KING|1125 W. NC HWY 54...|
|NC_Durham_97837|   NC|Durham|    97837|HAMPTON INN & SUITES|   1542 N GREGSON ST|
|NC_Durham_60690|   NC|Durham|    60690|BETTER LIVING CON...|       909 GARCIA ST|
+---------------+-----+------+---------+--------------------+--------------------+
only showing top 5 rows

root
 |-- datasetId: string (nullable = true)
 |-- fields: struct (nullable = true)
 |    |-- closing_date: string (nullable = true)
 |    |-- est_group_desc: string (nullable = true)
 |    |-- geolocation: array (nullable = tru

In [4]:
"""
   Union of two dataframes. ch 3, lab 230

   @author rambabu.posa
"""
import os
import util
from pyspark.sql import SparkSession

def get_absolute_file_path(path, filename):
    # To get absolute path for a given filename
    #current_dir = os.path.dirname(__file__)
    current_dir = '/home/vk/Spark_Source_Code/net.jgp.books.spark.ch03/src/main/python/lab230_dataframe/'
    relative_path = "{}{}".format(path, filename)
    absolute_file_path = os.path.join(current_dir, relative_path)
    return absolute_file_path

def main(spark):
    # The processing code.
    filename1 = 'Restaurants_in_Wake_County_NC.csv'
    path1 = '../../../../data/'
    absolute_file_path1 = get_absolute_file_path(path1, filename1)

    filename2 = 'Restaurants_in_Durham_County_NC.json'
    path2 = '../../../../data/'
    absolute_file_path2 = get_absolute_file_path(path2, filename2)

    df1 = spark.read.csv(path=absolute_file_path1, header=True, inferSchema=True)

    df2 = spark.read.json(absolute_file_path2)

    wakeRestaurantsDf = util.build_wake_restaurants_dataframe(df1)
    durhamRestaurantsDf = util.build_durham_restaurants_dataframe(df2)

    util.combineDataframes(wakeRestaurantsDf, durhamRestaurantsDf)

if __name__ == '__main__':
    # Creates a session on a local master
    spark = SparkSession.builder.appName("Union of two dataframes") \
        .master("local[*]").getOrCreate()

    # Comment this line to see full log
    spark.sparkContext.setLogLevel('warn')
    main(spark)
    spark.stop()

+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+-------+------------------+
| datasetId|                name|            address1|address2|       city|state|       zip|           tel|          dateStart|             type|        geoX|       geoY|county|dateEnd|                id|
+----------+--------------------+--------------------+--------+-----------+-----+----------+--------------+-------------------+-----------------+------------+-----------+------+-------+------------------+
|4092016024|                WABA|2502 1/2 HILLSBOR...|    NULL|    RALEIGH|   NC|     27607|(919) 833-1710|2011-10-18 04:00:00|       Restaurant|-78.66818477|35.78783803|  Wake|   NULL|NC_Wake_4092016024|
|4092021693|  WALMART DELI #2247|2010 KILDAIRE FAR...|    NULL|       CARY|   NC|     27518|(919) 852-6651|2011-11-08 04:00:00|       Food Stand|-78.78211173|35.73717591|  Wake|   

In [4]:
"""
  Converts an array to a Dataframe of strings.

   @author rambabu.posa
"""
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType


def main(spark):
    data = [['Jean'], ['Liz'], ['Pierre'], ['Lauric']]

    """
    * data:    parameter list1, data to create a dataset
    * encoder: parameter list2, implicit encoder
    """
    schema = StructType([StructField('name', StringType(), True)])

    df = spark.createDataFrame(data, schema)
    df.show()
    df.printSchema()

if __name__ == '__main__':
    # Creates a session on a local master
    spark = SparkSession.builder.appName("Array to Dataframe") \
        .master("local[*]").getOrCreate()

    # Comment this line to see full log
    spark.sparkContext.setLogLevel('warn')
    main(spark)
    spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/18 14:44:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+------+
|  name|
+------+
|  Jean|
|   Liz|
|Pierre|
|Lauric|
+------+

root
 |-- name: string (nullable = true)



In [26]:
# ch 3 lab 320 author: Vadim Kisarov
import os
from pyspark.sql import SparkSession
import logging
import pandas

# main() must be before her invoke!
def main(spark):
    path = "/home/vk/Spark_Source_Code/net.jgp.books.spark.ch03/data/books.csv"
    df = spark.read.csv(header = True,  inferSchema = True, path = path)
    
    logging.warning("Spark to Pandas DataFrame. Here {} row in file".format(df.count()))
    df.show()
    df.printSchema()
    
    df_pandas = df.toPandas()
    df_pandas = df_pandas[:5][['id', 'authorId', 'title', 'releaseDate']]
    print(df_pandas)
    print("\n")
    
    releaseDate = df_pandas["releaseDate"]
    
    df_spark = spark.createDataFrame(df_pandas)
    logging.warning("Back from Pandas to Spark DataFrame. Here {} rows now".format(df_spark.count()))
    df_spark.show()
    df_spark.printSchema()
    
if __name__ == '__main__':
    spark = SparkSession.builder.appName("CSV to dataframe to dataset and back") \
        .master("local") \
        .getOrCreate()
    
    spark.sparkContext.setLogLevel('warn')
    main(spark)
    spark.stop()
    



+---+--------+--------------------+-----------+--------------------+
| id|authorId|               title|releaseDate|                link|
+---+--------+--------------------+-----------+--------------------+
|  1|       1|Fantastic Beasts ...|   11/18/16|http://amzn.to/2k...|
|  2|       1|Harry Potter and ...|    10/6/15|http://amzn.to/2l...|
|  3|       1|The Tales of Beed...|    12/4/08|http://amzn.to/2k...|
|  4|       1|Harry Potter and ...|    10/4/16|http://amzn.to/2k...|
|  5|       2|Informix 12.10 on...|    4/23/17|http://amzn.to/2i...|
|  6|       2|Development Tools...|   12/28/16|http://amzn.to/2v...|
|  7|       3|Adventures of Huc...|    5/26/94|http://amzn.to/2w...|
|  8|       3|A Connecticut Yan...|    6/17/17|http://amzn.to/2x...|
| 10|       4|Jacques le Fataliste|     3/1/00|http://amzn.to/2u...|
| 11|       4|Diderot Encyclope...|       NULL|http://amzn.to/2i...|
| 12|    NULL|   A Woman in Berlin|    7/11/06|http://amzn.to/2i...|
| 13|       6|Spring Boot in Ac...

+---+--------+--------------------+-----------+
| id|authorId|               title|releaseDate|
+---+--------+--------------------+-----------+
|  1|     1.0|Fantastic Beasts ...|   11/18/16|
|  2|     1.0|Harry Potter and ...|    10/6/15|
|  3|     1.0|The Tales of Beed...|    12/4/08|
|  4|     1.0|Harry Potter and ...|    10/4/16|
|  5|     2.0|Informix 12.10 on...|    4/23/17|
+---+--------+--------------------+-----------+

root
 |-- id: long (nullable = true)
 |-- authorId: double (nullable = true)
 |-- title: string (nullable = true)
 |-- releaseDate: string (nullable = true)



In [43]:
import pandas as pd
releaseDate = pd.Series(['11/18/16', '10/6/15', '4/23/17', '12/31/1670', '6/15/1999', 'NULL'])
releaseDate
date_list = []
for string in releaseDate:
    spl_arrow = string.split("/")
    if len(spl_arrow) > 1:
       year = '20' + spl_arrow[2] if len(spl_arrow[2]) == 2 else spl_arrow[2]
       month = '0' + spl_arrow[0] if len(spl_arrow[0]) == 1 else spl_arrow[0]
       day = '0' + spl_arrow[1] if len(spl_arrow[1]) == 1 else spl_arrow[1]
       print(f"{year}-{month}-{day}")
      
    
        

    

2016-11-18
2015-10-06
2017-04-23
1670-12-31
1999-06-15
